> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langchainV1-essentials](https://github.com/langchain-ai/lca-langchainV1-essentials).  
> All original credit belongs to the course authors.

# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

from bedrock import nova_pro as llm

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find the total on Frank Harris's last invoice, I need to first identify Frank Harris's customer ID. Once I have the customer ID, I can then query the invoices table to find the total of the most recent invoice for that customer. </thinking>\n"}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': "SELECT customer_id FROM customers WHERE name = 'Frank Harris' LIMIT 1"}, 'id': 'tooluse_529KyzzXQtuVxxvGezsx_A'}]
Tool Calls:
  execute_sql (tooluse_529KyzzXQtuVxxvGezsx_A)
 Call ID: tooluse_529KyzzXQtuVxxvGezsx_A
  Args:
    query: SELECT customer_id FROM customers WHERE name = 'Frank Harris' LIMIT 1
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.Opera

In [9]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find the titles, I need to query the database for the title column. Since the user did not specify which table to query, I will assume they want titles from a table that commonly contains titles, such as a 'books' or 'movies' table. I will start by checking if there is a 'books' table and query the titles from there. If that fails, I will try the 'movies' table. </thinking>\n"}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': 'SELECT title FROM books LIMIT 5'}, 'id': 'tooluse_3k7npP5wSuKXAQ7q3JcawQ'}]
Tool Calls:
  execute_sql (tooluse_3k7npP5wSuKXAQ7q3JcawQ)
 Call ID: tooluse_3k7npP5wSuKXAQ7q3JcawQ
  Args:
    query: SELECT title FROM books LIMIT 5
================================= Tool Message =================================
Name: execute_sql



## Add memory

In [10]:
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [13]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find the total on Frank Harris's last invoice, I need to first identify Frank Harris's customer ID. Once I have the customer ID, I can then query the invoices table to find the total of the most recent invoice for that customer. </thinking>\n"}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': "SELECT customer_id FROM customers WHERE name = 'Frank Harris' LIMIT 1"}, 'id': 'tooluse_yzi6_aMpSoe22jlXY6T5Sg'}]
Tool Calls:
  execute_sql (tooluse_yzi6_aMpSoe22jlXY6T5Sg)
 Call ID: tooluse_yzi6_aMpSoe22jlXY6T5Sg
  Args:
    query: SELECT customer_id FROM customers WHERE name = 'Frank Harris' LIMIT 1
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.Opera

In [14]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find the titles of the items on Frank Harris's last invoice, I need to first identify the invoice ID of the most recent invoice. Once I have the invoice ID, I can then query the `InvoiceLine` table to find the titles of the items on that invoice. </thinking> "}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': "SELECT InvoiceId FROM Invoice WHERE CustomerId IN (SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris') ORDER BY InvoiceDate DESC LIMIT 1"}, 'id': 'tooluse_h9I0NMj9QHyfIucKtjSycg'}]
Tool Calls:
  execute_sql (tooluse_h9I0NMj9QHyfIucKtjSycg)
 Call ID: tooluse_h9I0NMj9QHyfIucKtjSycg
  Args:
    query: SELECT InvoiceId FROM Invoice WHERE CustomerId IN (SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND La

## Try your own queries
Now that there is memory, check the agents recall!

In [15]:
question = "Which is the customer with the highest sum of purchases?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Which is the customer with the highest sum of purchases?
================================== Ai Message ==================================

[{'type': 'text', 'text': '<thinking> To find the customer with the highest sum of purchases, I need to first sum the totals of all invoices for each customer. Then, I can order the results by the sum of totals in descending order and limit the results to the top row. </thinking> '}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': 'SELECT CustomerId, SUM(Total) AS TotalPurchases FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchases DESC LIMIT 1'}, 'id': 'tooluse_tPApyUD5RuKm0JjXnJgQgQ'}]
Tool Calls:
  execute_sql (tooluse_tPApyUD5RuKm0JjXnJgQgQ)
 Call ID: tooluse_tPApyUD5RuKm0JjXnJgQgQ
  Args:
    query: SELECT CustomerId, SUM(Total) AS TotalPurchases FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchases DESC LIMIT 1
===============================

In [16]:
question = "Which was her highest purchase?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

Which was her highest purchase?
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking> To find Helena Holý's highest purchase, I need to first identify her customer ID. Once I have the customer ID, I can then query the `Invoice` table to find the highest total of her invoices. </thinking> "}, {'type': 'tool_use', 'name': 'execute_sql', 'input': {'query': "SELECT CustomerId FROM Customer WHERE FirstName = 'Helena' AND LastName = 'Holý' LIMIT 1"}, 'id': 'tooluse_g7XW3rdNSySlMyowyPwDNA'}]
Tool Calls:
  execute_sql (tooluse_g7XW3rdNSySlMyowyPwDNA)
 Call ID: tooluse_g7XW3rdNSySlMyowyPwDNA
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Helena' AND LastName = 'Holý' LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(6,)]
================================== Ai Message =